# Import packages

In [1]:
import pandas as pd
from teradataml import *
import ast
import json
import torch
from teradataml.dataframe.copy_to import copy_to_sql
from transformers.generation import GenerationMixin

from modules.db import TeradataDatabase
from modules.models import( 
    OpusTranslationModelConfig, 
                        OpusTranslationModel, 
                        SentenceEmbeddingConfig, 
                        SentenceEmbeddingModel, 
)
from utils import clean_text, load_embedding_model, unicode_clean, load_translation_model
from constants import (
    CLEANED_TEST_DATA_PATH, TRAIN_VAL_DATA_PATH, CLASS_EMBEDDINGS_PATH, PRODUCT_TEST_EMBEDDINGS_PATH, 
    CLEANED_GPC_PATH, CLEANED_TEST_DATA_PATH, TEST_DATA_PATH, E5_LARGE_INSTRUCT_CONFIG_PATH, 
    OPUS_TRANSLATION_CONFIG_PATH, DATA_PATH, PRODUCT_TRAIN_EMBEDDINGS_PATH, VALIDATION_DATA_PATH,
    FULL_DATA_SET_DATA_PATH, PRODUCT_FULL_DATASET_EMBEDDINGS_PATH,  CLEANED_FULL_DATASET_DATA_PATH,
    CLASS_EMBEDDINGS_PATH_QWEN, PRODUCT_FULL_DATASET_EMBEDDINGS__QWEN_PATH,
    MWPD_DATA_SET_TRAIN, MWPD_DATA_SET_VAL, MWPD_DATA_SET_TEST, MWPD_FULL_DATASET
)


## Connect to database

In [3]:
td_db = TeradataDatabase()
td_db.connect()

In [ ]:
tdf = td_db.execute_query("Select * from demo_user.gpc_orig")
df = pd.DataFrame(tdf)


In [ ]:
df.head()

### Remvoing un-neccesary columns

In [ ]:
query = """
ALTER TABLE demo_user.gpc_orig 
DROP SegmentCode,
DROP FamilyCode,
DROP ClassCode,
Drop BrickCode,
DROP AttributeCode,
DROP AttributeTitle,
DROP AttributeDefinition,
DROP AttributeValueCode,
DROP AttributeValueTitle,
DROP AttributeValueDefinition;
"""

In [11]:
tdf = td_db.execute_query(query)

### Cleaning the GPC Columns

In [ ]:
cleaning_query = """
UPDATE demo_user.gpc_orig
SET SegmentTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(SegmentTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    SegmentDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(SegmentDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    FamilyTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(FamilyTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    FamilyDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(FamilyDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    ClassTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(ClassTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    ClassDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(ClassDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickDefinition_Includes = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickDefinition_Includes, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickDefinition_Excludes = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickDefinition_Excludes, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' ')));

"""

In [13]:
tdf = td_db.execute_query(cleaning_query)

## Disconnect

In [ ]:
td_db.disconnect()